In [9]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv('requests1.csv')

In [10]:
print("Tổng số lượng requests tới server: \n", df.count())

Tổng số lượng requests tới server: 
 Timestamp        11874
Status           11874
Response Time    11874
Client IP        11874
Method           11874
Endpoint         11874
dtype: int64


In [12]:
print("Số lượng các Method: ", df["Method"].nunique())
print(df["Method"].unique())

Số lượng các Method:  3
['GET' 'POST' 'HEAD']


In [13]:
print("Số lượng các API Endpoint: ", df["Endpoint"].nunique())
print(df["Endpoint"].unique())

Số lượng các API Endpoint:  9
['/api/tags' '/api/version' '/api/chat' '/api/show' '/api/generate'
 '/api/embed' '/' '/favicon.ico' '/api/ps']


In [ ]:
# Thống kê số lượng tin nhắn của mỗi thành viên
count = df['Client IP'].nunique()
message_count = df['Client IP'].value_counts(ascending=True)

# Xuất kết quả
print("Số lượng IP:", count)
print("Thống kê số lần gửi request:")
print(message_count)

# # Lưu kết quả vào file Excel (tùy chọn)
# message_count.to_excel('message_statistics.xlsx')